## Part A: initiate the model ingredients and define functions (to be used in the optimization step).

Load modules and packages:

In [78]:
using Random, Distributions, JuMP

### Step 1: Set the parameter values.

In [79]:
N = 3 # Number of countries
J = 10 # Number of goods (needs to be big number and an integer type)
theta = 4 # Frechet shape parameter (governs comparative advantage)
T = ones(N, 1) * 1.5 # Frechet scale parameter (governs absolute advantage)
sigma = 2 # Substitution elasticity between goods
tau = ones(N, N) # Iceberg trade costs
L = ones(N, 1) # Size of labor force in each country

Random.seed!(1) # Seed for the random number generator (to guarantee reproducibility; this is standard in research these days)

TaskLocalRNG()

### Step 2 : Draw a $J$-by-$N$ matrix of uniform random variables on $[0,1]$ and convert them to Fréchet random variables using the inverse CDF.

These are the productivities for each good in each country.

Note: In the 'Distributions' package, the inverse CDF can be calculated using the 'quantile()' function for any distribution defined in the package, of which 'Frechet' is one. More on the 'Distributions' package is available at https://juliastats.org/Distributions.jl/v0.14/index.html.

Since you will be comparing economies with different shape parameters ($\theta$), you can write a simple function where $\theta$ is an argument to transform the uniform draws into Fréchet-distributed productivities.

Save the productivities in an array. You can call it $z$.

Let $n \in {1,2,3}$ index the three countries and WLOG let $n=1$ be the "Home" country. Normalize the wage in "Home" to $w = 1$. The wages in countries 2 and 3 are endogenous variables.

### Step 3: Create a function that computes the trades shares for all countries, given the productivities and wages.

Calculating these shares involves a couple of substeps:
1. identify the lowest-cost producer of good $j$ in each of the $N$ different countries
1. compute the trade shares given the selection in the previous sub-step.

Note: Even though the wages in countries 2 and 3 are endogenous, you need to use a place holder for the purposes of defining the function. The function accepts these wages as arguments and in the optimization step the wages will be such that trade is balanced at the global scale.

### Step 4: Create a function that computes the trade balance for each country, again for given productivities and wages.

Note: Use the shares and country-specific incomes to compute bilateral trade balances and then sum them across countries.

### Step 5: Create a function that reports some additional equilibrium results.

The functions you have defined so far will be used in a non-linear optimizer that solves the system of equations. Let the output of this optimization be 'sol' and treat it as a placeholder for now. In the optimization step 'sol' will be the vector of wages in all N countries.

In particular, store the following results in arrays:
1. equilibrium wages, and
2. welfare (i.e., $\tfrac{wL}{P}$).

You can also print these results to the screen using the 'println' function or command.

# Part B: solve the model

### Question a.
Countries are symmetric in terms of size and productivities. Trade is free (i.e., the iceberg costs of international trade between any pair of origin and destination are normalized to 1). Symmetry implies that wages are equalized across countries, provided that J is large enough and therefore that law or large numbers applies. In other words, we don't actually have to solve for the wage vector and we could instead compute the trade shares for a give vector of unit-value wages. Also, since international trade is free, the expenditure shares $\pi$ calculated using the 'lcp' function are symmetric as well (namely, $\pi$ is an $N \times N$ matrix with identical elements equal to $\tfrac{1}{N}$)

Although we already know the correct answer to this problem, we will still solve for the equilibrium wages using a numerical algorithm. This will allow us to verify that our functions and the solution algorithm are working correctly. We can then solve the other cases 

There are several options for solving systems of non-linear equations numerically. In this particular case, we are looking for the root of the sum of country-specific trade balances (in other words, the vector of wages such that the *global* trade balance equals zero). In other work, I have used the *JuMP.jl* package to solve systems of equations. The package has advantages and drawbacks. The advantage is that it can call a number of different solvers and it can handle a variety of optimization problems (constrained, unconstrained, linear, non-linear,...). The drawback is that the syntax has to fit the requirements of the package. I think this simple Eaton-Kortum problem is a "nice" sandbox for you to get your hands dirty with with this optimization toolkit. The *julia.quantecon.org* website has some helpful material and references to get you started in chapter 9.

### Question b.

Countries remain symmetric but international trade is subject to uniform iceberg costs $\tau = 1.1$.


### Question c.
Countries are still symmetric but country 3 is "remote", i.e. $\tau_{13} = \tau_{31} = \tau_{23} = \tau_{32} = 1.3$. The other trade costs are the same as in b.

### Question d.
Country 2 is more productive than 1 and 3. Everything else is as in c.

### Question e.
Change the value of the shape parameter in the Fréchet distribution but keep everything else as in c.